In [17]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]
basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

end data preparation, 
the new data represent the item who has been bought together.

In [24]:
basket.head()

Description   50'S CHRISTMAS GIFT BAG LARGE   DOLLY GIRL BEAKER  \
InvoiceNo                                                         
536370                                  0.0                 0.0   
536852                                  0.0                 0.0   
536974                                  0.0                 0.0   
537065                                  0.0                 0.0   
537463                                  0.0                 0.0   

Description   I LOVE LONDON MINI BACKPACK   NINE DRAWER OFFICE TIDY  \
InvoiceNo                                                             
536370                                0.0                       0.0   
536852                                0.0                       0.0   
536974                                0.0                       0.0   
537065                                0.0                       0.0   
537463                                0.0                       0.0   

Description   SET 2 TEA TOWELS I LOVE LONDON    SPACEBOY BABY GIFT SET  \
InvoiceNo                                                                
536370                                   24.0                      0.0   
536852                                    0.0                      0.0   
536974                                    0.0                      0.0   
537065                                    0.0                      0.0   
537463                                    0.0                      0.0   

Description   TRELLIS COAT RACK  10 COLOUR SPACEBOY PEN  \
InvoiceNo                                                 
536370                      0.0                     0.0   
536852                      0.0                     0.0   
536974                      0.0                     0.0   
537065                      0.0                     0.0   
537463                      0.0                     0.0   

Description  12 COLOURED PARTY BALLOONS  12 EGG HOUSE PAINTED WOOD  ...  \
InvoiceNo                                                           ...   
536370                              0.0                        0.0  ...   
536852                              0.0                        0.0  ...   
536974                              0.0                        0.0  ...   
537065                              0.0                        0.0  ...   
537463                              0.0                        0.0  ...   

Description  WRAP VINTAGE PETALS  DESIGN  YELLOW COAT RACK PARIS FASHION  \
InvoiceNo                                                                  
536370                               0.0                             0.0   
536852                               0.0                             0.0   
536974                               0.0                             0.0   
537065                               0.0                             0.0   
537463                               0.0                             0.0   

Description  YELLOW GIANT GARDEN THERMOMETER  YELLOW SHARK HELICOPTER  \
InvoiceNo                                                               
536370                                   0.0                      0.0   
536852                                   0.0                      0.0   
536974                                   0.0                      0.0   
537065                                   0.0                      0.0   
537463                                   0.0                      0.0   

Description  ZINC  STAR T-LIGHT HOLDER   ZINC FOLKART SLEIGH BELLS  \
InvoiceNo                                                            
536370                              0.0                        0.0   
536852                              0.0                        0.0   
536974                              0.0                        0.0   
537065                              0.0                        0.0   
537463                              0.0                        0.0   

Description  ZINC HERB GARDEN CONTAIN

In [19]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [21]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [22]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]


antecedents  \
10                      (SET/6 RED SPOTTY PAPER CUPS)   
11                    (SET/6 RED SPOTTY PAPER PLATES)   
12  (SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...   
13  (SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...   
14  (SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...   

                              consequents  antecedent support  \
10        (SET/6 RED SPOTTY PAPER PLATES)            0.117137   
11          (SET/6 RED SPOTTY PAPER CUPS)            0.108460   
12  (SET/20 RED RETROSPOT PAPER NAPKINS )            0.104121   
13        (SET/6 RED SPOTTY PAPER PLATES)            0.086768   
14          (SET/6 RED SPOTTY PAPER CUPS)            0.086768   

    consequent support   support  confidence      lift  leverage  conviction  
10            0.108460  0.104121    0.888889  8.195556  0.091417    8.023861  
11            0.117137  0.104121    0.960000  8.195556  0.091417   22.071584  
12            0.112798  0.084599    0.812500  7.203125  0.072854    4.731743  
13            0.108460  0.084599    0.975000  8.989500  0.075188   35.661605  
14            0.117137  0.084599    0.975000  8.323611  0.074435   35.314534